In [0]:
import zipfile
import os

# Path where the ZIP was uploaded (replace with yours from upload cell)
zip_path = "/Volumes/ambulance_project/ambulance_data/raw_data/EMS_Incident_Dispatch_Data.csv.zip"
extract_path = "/Volumes/ambulance_project/ambulance_data/raw_data"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted to:", extract_path)

In [0]:
# Replace file names with the exact names you have
nta_path = "/Volumes/ambulance_project/ambulance_data/raw_data/NTAs_Neighborhood_Tabulation_Areas.csv"
dot_path = "/Volumes/ambulance_project/ambulance_data/raw_data/DOT_Traffic_Speeds_NBE_20250807.csv"
ems_path = "/Volumes/ambulance_project/ambulance_data/raw_data/EMS_Incident_Dispatch_Data.csv"

nta_df = spark.read.format("csv").option("header", "true").load(nta_path)
dot_df = spark.read.format("csv").option("header", "true").load(dot_path)
ems_df = spark.read.format("csv").option("header", "true").load(ems_path)

# Register as SQL views
nta_df.createOrReplaceTempView("nta")
dot_df.createOrReplaceTempView("dot")
ems_df.createOrReplaceTempView("ems")

In [0]:
nta_df.head()

In [0]:
dot_df.head()

In [0]:
ems_df.head()

In [0]:
# View schema (column names + types)
nta_df.printSchema()
dot_df.printSchema()
ems_df.printSchema()

In [0]:
# Count rows
print("NTA rows count:", nta_df.count())
print("DOT rows count:", dot_df.count())
print("EMS rows count:", ems_df.count())

DOT CLEANING

In [0]:
# from pyspark.sql import functions as F
# from pyspark.sql.window import Window

# dot_raw = spark.table("ambulance_project.ambulance_data.t_dot")

# # Keep the useful columns (others are not needed for prediction/joining)
# keep_cols = [c for c in dot_raw.columns if c in {
#     "ID","SPEED","TRAVEL_TIME","DATA_AS_OF","LINK_ID","LINK_POINTS","BOROUGH","LINK_NAME"
# }]
# dot = dot_raw.select(*keep_cols)

# dot = (
#     dot
#     .withColumn("dot_id", F.col("ID").cast("long"))
#     .withColumn("link_id", F.col("LINK_ID").cast("long"))
#     .withColumn("speed_mph", F.col("SPEED").cast("double"))
#     .withColumn("travel_time_sec", F.col("TRAVEL_TIME").cast("double"))
#     .withColumn("as_of_ts", F.to_timestamp(F.col("DATA_AS_OF"), "MM/dd/yyyy hh:mm:ss a"))
#     .withColumn("borough", F.upper(F.trim(F.col("BOROUGH"))))
#     .withColumn("link_points", F.regexp_replace(F.trim(F.col("LINK_POINTS")), r"\s+", " "))  # normalize spaces
# )

# # ---- SAFE parsing of start/end coordinates from LINK_POINTS ----
# # LINK_POINTS looks like: "40.7081105,-73.99944 40.7084705,-73.99884 ..."
# points = F.split(F.col("link_points"), " ")

# first_p = F.element_at(points, 1)          # 1-based; NULL-safe
# last_p  = F.element_at(points, -1)         # last element; NULL-safe

# first_split = F.split(first_p, ",")
# last_split  = F.split(last_p, ",")

# start_lat = F.element_at(first_split, 1).cast("double")
# start_lon = F.element_at(first_split, 2).cast("double")
# end_lat   = F.element_at(last_split, 1).cast("double")
# end_lon   = F.element_at(last_split, 2).cast("double")

# dot = dot.withColumn("start_lat", start_lat)\
#          .withColumn("start_lon", start_lon)\
#          .withColumn("end_lat",   end_lat)\
#          .withColumn("end_lon",   end_lon)

# # Representative midpoint for spatial join / gridding
# dot = dot.withColumn(
#     "mid_lat",
#     F.when(F.col("start_lat").isNotNull() & F.col("end_lat").isNotNull(),
#            (F.col("start_lat")+F.col("end_lat"))/2.0).otherwise(F.col("start_lat"))
# ).withColumn(
#     "mid_lon",
#     F.when(F.col("start_lon").isNotNull() & F.col("end_lon").isNotNull(),
#            (F.col("start_lon")+F.col("end_lon"))/2.0).otherwise(F.col("start_lon"))
# )

# # Quality filters
# dot = dot.filter((F.col("speed_mph") >= 0) & (F.col("speed_mph") <= 120))\
#          .filter(F.col("travel_time_sec") >= 0)

# # Keep latest record per link_id (DOT feeds are time-series by link)
# w = Window.partitionBy("link_id").orderBy(F.col("as_of_ts").desc_nulls_last())
# dot_latest = dot.withColumn("rn", F.row_number().over(w)).filter("rn = 1").drop("rn","ID","SPEED","TRAVEL_TIME","DATA_AS_OF","LINK_POINTS")

# # Save a clean table
# dot_latest.write.mode("overwrite").saveAsTable("ambulance_project.ambulance_data.ct_dot")

# display(spark.table("ambulance_project.ambulance_data.ct_dot").limit(10))


In [0]:
dot_df.show(5)